# Nature Based Solution What-if scenario with SFINCS

**This notebook illustrates the set up, execution, and visualization of an inundation model to investigate Nature based Solutions for coastal flooding. The example here presents a usecase for the Wash (United Kingdom) and combines input data from multiple sources.**

_Technical information: 
The model is set up using the HydroMT_sfincs python package. The usecase here requires a selection of its functionalities. If you want to learn more about building sfincs models this way, visit [HydroMT_sfincs - Modelbuilder example from script](https://github.com/Deltares/hydromt_sfincs/blob/main/examples/build_from_script.ipynb)._

_This modelbuilder combines many other Python packages. HydroMT_sfincs is used for the set up of the sfincs model, matplotlib for visualization, (rio)xarray to process netcdf files and geopandas to process geospatial data._

# User Input

**_Users can change the grid resolution, roughness parameters and simulate sea level rise what-if scenarios._**

In [1]:
# Define the grid resolution
grid_resolution     = 50     # meters

# Define the roughness parameters
manning_saltmarshes = 0.08   # 0.07 Rezaie et al., 2020
manning_mangroves   = 0.15   # Zhang et al., 2012
manning_land        = 0.04   # default
manning_sea         = 0.02   # default

# Define waterlevel offset
slr_mean = 0.0

In [ ]:
# Connect to EDITO
from edito_process_api import set_edito_token
tok = set_edito_token()   # prompts username/password, sets EDITO_ACCESS_TOKEN
print("✅ EDITO token set | len:", len(tok))

# Workflow

In [ ]:
from IPython.display import HTML
HTML("""
<style>
/* JupyterLab: hide inputs of CODE cells only (keep Markdown visible) */
.jp-Notebook .jp-CodeCell .jp-InputArea { display: none !important; }
.jp-Notebook .jp-CodeCell .jp-InputPrompt { display: none !important; }

/* Classic Notebook: hide inputs of CODE cells only */
div.code_cell > div.input { display: none !important; }
div.code_cell .prompt { display: none !important; }
</style>
""")


## Imports and variables

In [ ]:
from shapely.geometry import Point
from os.path import join
import geopandas as gpd
import hydromt
from hydromt_sfincs import SfincsModel
import os
import shutil

In [ ]:
# Define the main directory and data library
# This is the directory where the data library is stored
data_libs = './input_dir/edito_sfincs_data.yml'
input_dir = './input_dir' 
main_dir = './wash_uk_example' 

# This is the directory where the model will be saved
sim_dir   = join(main_dir, 'sims')
if not os.path.exists(sim_dir):
    os.makedirs(sim_dir)

In [ ]:
# Define the input files
fn_domain       =  join(input_dir, 'domain.gpkg')
fn_domain_lines =  join(input_dir, 'domain_lines.gpkg')
fn_topo         = join(input_dir, 'delta_dtm_gebco_ref_msl.tif')
fn_veg          =  join(input_dir, 'da_veg.tif')
fn_storm_wl     =  join(input_dir, 'wl_ts.nc')

In [ ]:
# Define the model observation points
obs_names             = ['veg1','veg2']
obs_pnts              = [Point(303082.833,5864791.153), 
                         Point(315739.571,5854677.962)]
print("✅ Input files defined.")

## Initialization

In [ ]:
# Initialize hydromt DataCatalog
data_cat  = hydromt.DataCatalog(data_libs = data_libs)

# Initialize SFINCS model
mod = SfincsModel(root  = sim_dir+ '/sim_veg',  mode = 'w+', data_libs = data_libs)
gdf_domain       = gpd.read_file(fn_domain)
gdf_domain_lines = gpd.read_file(fn_domain_lines) 
print("✅ SFINCS model initialized.")

## Grid & Topography

In [ ]:
# setup grid from polygon in GeoDataFrame   
mod.setup_grid_from_region(region = {'geom':gdf_domain}, res = grid_resolution, rotated = True)
mod.config

# Define topography and vegetation datasets
da_topo   = data_cat.get_rasterdataset(fn_topo)
da_veg    = data_cat.get_rasterdataset(fn_veg)

# Assign topography to the model
datasets_dep  = [{"elevtn":da_topo}]
da_dep        = mod.setup_dep(datasets_dep=datasets_dep, buffer_cells= 1) # 

# set mask based on elevation (areas in km2)
mod.setup_mask_active(mask = gdf_domain , zmax = 10, drop_area = 10, fill_area = 100, reset_mask = True) 
mod.set_config("stopdepth", '250.0')
   
# Assign observation points to the model
gdf_obs               = gpd.GeoDataFrame({'names': obs_names}, geometry = obs_pnts, crs = mod.crs).set_index('names')
mod.setup_observation_points(gdf_obs)
print("✅ Grid and topography defined.")

## Boundary Conditions

In [ ]:
# Define model boundaries
gdf_all_lines = gdf_domain_lines.loc[(gdf_domain_lines['bnd']==1)] 
if len(gdf_all_lines) > 0:
    gdf_all_lines = gdf_all_lines.loc[gdf_all_lines['geometry'] != None]
    gdf_all_lines.geometry = gdf_all_lines['geometry'].buffer(0.0075)
mod.setup_mask_bounds(btype = "waterlevel", include_mask = gdf_all_lines  , reset_bounds=True, zmax = 1) # here we mask the wl for all bnd points     

#update times, in line with wl forcing (times should be updated afeter setup_mask_bounds)
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", "20000101 000000")
mod.set_config("tstop" , "20000106 000000")
mod.set_config("dtout",  "600")
print("✅ Boundary conditions defined.")

## Forcing

In [ ]:

mod.setup_waterlevel_forcing(geodataset = fn_storm_wl, offset=slr_mean , buffer = 15000)
df_ts = mod.forcing['bzs'].to_dataframe()
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0]
df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1]
t_start = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[0][0]).replace('-','').replace(':','')
t_end   = str(df_ts.index[~df_ts.isnull().any(axis=1)].tolist()[-1][0]).replace('-','').replace(':','')

#update times, in line with wl forcing
mod.set_config("tref"  , "20000101 000000")
mod.set_config("tstart", t_start)
mod.set_config("tstop" , t_end)
mod.set_config("dtout",  "600")
mod.set_config("dtmaxout","9999999")
print("✅ Forcing defined.")

## Add spatially varying roughness data

To incorporate the influence of different roughness for land, sea, and vegetation on flow this is added to the model based on our data.

In [ ]:
# set increased Manning values for vegetation cover
mapping = {1:manning_saltmarshes, 2:manning_mangroves}
manning_veg_map = da_veg.copy()
for old_value, new_value in mapping.items():
    manning_veg_map = manning_veg_map.where(manning_veg_map != old_value, new_value)
  
datasets_rgh = [{'manning':manning_veg_map}]
mod.setup_manning_roughness(
           datasets_rgh=datasets_rgh,
           manning_land= manning_land,
           manning_sea= manning_sea,
           rgh_lev_land=0,  # the minimum elevation of the land
       )
print("✅ Spatially varying roughness data defined.")

## Write to file

In [ ]:
# Write the model with vegation to files
mod.write()

# create a copy for the model without vegetation for comparison
shutil.copytree(sim_dir + '/sim_veg', sim_dir + '/sim_noveg', dirs_exist_ok = True)
os.remove(sim_dir + '/sim_noveg/hydromt_data.yml')

# load the model and change manning file 
mod_noveg = SfincsModel(root  = sim_dir + '/sim_noveg',  mode = 'r+', data_libs = data_libs)
mod_noveg.config

mod_noveg.setup_manning_roughness(
              manning_land = manning_land,
              manning_sea  = manning_sea,
              rgh_lev_land = 0)  # the minimum elevation of the land

mod_noveg.write()
print("✅ Model configurations written.")

## Upload model to EDITO storage [no vegetation]

In [ ]:
# upload the generated model to youe s3 bucket in "SFINCS_INPUT"
from upload_model import upload_model_to_s3_bucket
upload_model_to_s3_bucket(join(sim_dir, 'sim_noveg'), 'SFINCS_INPUT')

## Run SFINCS simulation [no vegetation]
**Choose between SFINCS CPU or GPU versions**

In [ ]:
from edito_process_api import EditoClient, build_s3_from_env
client = EditoClient()
job_url, final_job, results = client.submit_and_watch(
    "process-playground-sfincs-run-docker-gpu-0.2.3",
    metadata={},
    process_inputs={
        "onyxia": {"friendlyName": "sfincs-gpu", "share": False},
        "resources": {
            "requests": {"cpu": "1000m", "memory": "4Gi"},
            "limits":   {"cpu": "4000m", "memory": "16Gi", "nvidia.com/gpu": "1"},
        },
        "s3": build_s3_from_env(),
    },
)


## Import simulation output [no vegetation]

In [ ]:
import time
from edito_process_api import EditoClient
from download_from_s3 import download_nc_files_from_s3

client = EditoClient()

# 1) wait for completion
final_job = client.wait_until_success(job_url, poll_seconds=5, timeout_seconds=7200)

# 2) download outputs
download_nc_files_from_s3(
    files_to_download=["sfincs_map.nc", "sfincs_his.nc"],
    s3_subfolder="SFINCS_OUTPUT",
    local_output_dir=join(sim_dir, "sim_noveg"),  # or "sim_veg"
)
print("✅ Download complete.")


## Upload model to EDITO storage [vegetation]

In [ ]:
from pathlib import Path
from os.path import join
from typing import List

# your uploader
from upload_model import upload_model_to_s3_bucket

# ---- configure per run ----
SIM_DIR          = sim_dir                     # assume you already defined this elsewhere
NOVEG_DIR        = join(SIM_DIR, "sim_noveg")  # where you downloaded model outputs
VEG_DIR          = join(SIM_DIR, "sim_veg")    # the directory to upload as the next model
REQUIRED_FILES   = ["sfincs_map.nc", "sfincs_his.nc"]
S3_TARGET_PREFIX = "SFINCS_INPUT"              # destination subfolder

def all_present_nonempty(folder: str, names: List[str]) -> bool:
    ok = True
    for n in names:
        p = Path(folder) / n
        if not p.exists():
            print(f"❌ missing: {p}")
            ok = False
        elif p.stat().st_size == 0:
            print(f"❌ empty file: {p}")
            ok = False
        else:
            print(f"✅ found: {p}  ({p.stat().st_size} bytes)")
    return ok

# 1) verify sim_noveg was downloaded correctly
print(f"Checking downloads in: {NOVEG_DIR}")
if not all_present_nonempty(NOVEG_DIR, REQUIRED_FILES):
    raise RuntimeError("Download in sim_noveg is incomplete — aborting upload step.")

# 2) sanity check that sim_veg exists and is not empty (to avoid uploading an empty folder)
veg_paths = list(Path(VEG_DIR).glob("*"))
if not veg_paths:
    raise RuntimeError(f"'{VEG_DIR}' is empty or missing — nothing to upload.")

# (optional) show what will be uploaded
print(f"Uploading {len(veg_paths)} items from {VEG_DIR} → s3://<your-bucket>/{S3_TARGET_PREFIX}/")

# 3) upload sim_veg ONLY after sim_noveg verified
upload_model_to_s3_bucket(VEG_DIR, S3_TARGET_PREFIX)
print("✅ Upload complete.")


## Run SFINCS simulation [vegetation]
**Choose between SFINCS CPU or GPU versions**

In [ ]:
from edito_process_api import EditoClient, build_s3_from_env
client = EditoClient()
job_url, final_job, results = client.submit_and_watch(
    "process-playground-sfincs-run-docker-gpu-0.2.3",
    metadata={},
    process_inputs={
        "onyxia": {"friendlyName": "sfincs-gpu", "share": False},
        "resources": {
            "requests": {"cpu": "1000m", "memory": "4Gi"},
            "limits":   {"cpu": "4000m", "memory": "16Gi", "nvidia.com/gpu": "1"},
        },
        "s3": build_s3_from_env(),
    },
)


## Import simulation output [vegetation]

In [ ]:
# ---- use it (assuming you already have job_url) ----
final_job = client.wait_until_success(job_url, poll_seconds=5, timeout_seconds=7200)

# proceed: download outputs
download_nc_files_from_s3(
    files_to_download=["sfincs_map.nc", "sfincs_his.nc"],
    s3_subfolder="SFINCS_OUTPUT",
    local_output_dir=join(sim_dir, "sim_veg"),  # or "sim_veg"
)
print("✅ Download complete.")

## Visualize output

In [ ]:
from sfincs_utils import load_hmax_noveg, load_h_his
from plot_utils import plot_snapshots_every_nth
from movie_utils import make_movie_from_pngs

# Setup
dir_noveg = join(sim_dir, 'sim_noveg')
dir_veg = join(sim_dir, 'sim_veg')
out_dir = join(sim_dir, 'figs/dif')
out_dir_his = join(sim_dir, 'figs')
os.makedirs(out_dir, exist_ok=True)

# Load model output
hmin = 0.05
fs_ticks = 9
da_h_dif, da_h_noveg, mod = load_hmax_noveg(dir_noveg, dir_veg, hmin)
his_noveg, his_veg = load_h_his(dir_noveg, dir_veg)
print("✅ Model outputs loaded.")

## Plot water level forcing

In [ ]:
from plot_utils import plot_waterlevel_forcing

# Plot water level forcing
plot_waterlevel_forcing(
    input_path=input_dir,
    output_path=join(out_dir_his, 'forcing_wl.png')
)

## Plot time series difference

In [ ]:
from plot_utils import plot_observed_h_difference

plot_observed_h_difference(
    his_noveg=his_noveg,
    his_veg=his_veg,
    output_path=join(out_dir_his, 'obs_dif.png')
)

## Create video

In [ ]:
# Apply GSWO mask
gswo = data_cat.get_rasterdataset(join(input_dir, 'gswo.tif'))
gswo_mask = gswo.raster.reproject_like(mod.grid, method="max") <= 5
da_h_noveg = da_h_noveg.where(gswo_mask).where(da_h_noveg > hmin)
da_h_dif = da_h_dif.where(abs(da_h_dif) > hmin / 2)

# Only plot every nth timestep
plot_snapshots_every_nth(
    da_h_noveg=da_h_noveg,
    da_h_dif=da_h_dif,
    his_noveg=his_noveg,
    out_dir=out_dir,
    n=20
)

# Create movie
make_movie_from_pngs(out_dir, fps=4, show_first_frame=True)

## Upload plots to EDITO storage

In [ ]:
#Copy movie to figs/
movie_src = join(sim_dir, 'figs', 'dif', 'output_movie.mp4')
movie_dst = join(sim_dir, 'figs', 'output_movie.mp4')
shutil.copy(movie_src, movie_dst)

#Upload figures and movie
upload_model_to_s3_bucket(join(sim_dir, 'figs'), 'SFINCS_OUTPUT', clean_s3=True)